In [1]:
# prep for notebook
import sys
from pathlib import Path
from pprint import pprint

# 添加项目根目录到路径
notebook_dir = Path().resolve()     # test 目录
project_root = notebook_dir.parent  # 项目根目录
sys.path.insert(0, str(project_root))

import os
os.chdir("E:\\Workshop\\SC-Flint\\sc-cn-desc-generator")

In [13]:
import re
import json
import logging
import yaml

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import src.config as config
from module.replacer import DescTemplateReplacer

with open('config.yaml', 'r', -1, config.ENCODE) as file:
    config_data = yaml.load(file, Loader=yaml.FullLoader)
    project_id = config_data['paratranz']['proj_id']
    auth_token = config_data['paratranz']['token']

In [4]:

'''
stage:
0  - 未翻译
1  - 已翻译
2  - 有疑问
3  - 已检查
5  - 已审核（二校）
9  - 已锁定，此状态下仅管理员可解锁，词条强制按译文导出
-1 - 已隐藏，此状态下词条强制按原文导出
page: 页面
pageSize: 每页大小

返回：
pageCount: 一共几页
'''
url = f"https://paratranz.cn/api/projects/{project_id}/strings?stage=0&pageSize=2"

headers = {'Authorization': auth_token}

response = requests.request("GET", url, headers=headers)
print(response.status_code)
pprint(response.json())

200
{'page': 1,
 'pageCount': 48,
 'pageSize': 2,
 'results': [{'comments': [],
              'context': '',
              'createdAt': '2025-12-11T07:51:47.000Z',
              'file': {'id': 2637743,
                       'name': 'test_paratranz_v0.0.0.json',
                       'project': 17306},
              'fileId': 2637743,
              'history': [],
              'id': 839578686,
              'importHistory': [],
              'key': 'item_DescFood_biscuit_1_a',
              'original': 'NDR: 10\\nEffects: Energizing, '
                          'Hyper-Metabolic\\n\\nThis holiday favorite cookie '
                          'features a crumbly shortbread ring filled with a '
                          'sweet and tart jam. Often enjoyed during Luminalia. '
                          '\\n',
              'project': 17306,
              'stage': 0,
              'translation': 'NDR：10\\n效果：充沛精力，加速代谢\\n\\n这款节日里最受欢迎的曲奇饼干的特点是酥脆的饼圈里填满了酸甜可口的果酱，经常在光灯节期间供人享用。\\n',
              'ui

In [6]:
url = f"https://paratranz.cn/api/projects/{project_id}/strings/{839578692}"

headers = {'Authorization': auth_token}

response = requests.request("GET", url, headers=headers)
print(response.status_code)
pprint(response.json())
print(response.json()['original'])

200
{'comments': [],
 'context': '',
 'createdAt': '2025-12-11T07:51:47.000Z',
 'file': {'id': 2637743,
          'name': 'test_paratranz_v0.0.0.json',
          'project': 17306},
 'fileId': 2637743,
 'history': [],
 'id': 839578692,
 'importHistory': [],
 'key': 'item_Desc_oct_pants_01_Shared',
 'original': 'Carrying Capacity: 1K µSCU\\n\\nOctagon embraces a classic look '
             'and makes it boldly modern with the Altimont. These jodhpur '
             'style pants feature a slightly flared hip that becomes snug '
             'under the knees thanks to built-in garters. A uniquely shaped '
             'front seam secured by silver clips on the waist and shin provide '
             "further detailing that's both subtle and stylish. ",
 'project': 17306,
 'stage': 0,
 'translation': '',
 'uid': None,
 'updatedAt': None,
 'version': 6193182,
 'words': 59}
Carrying Capacity: 1K µSCU\n\nOctagon embraces a classic look and makes it boldly modern with the Altimont. These jodhpur s

In [ ]:
url = f"https://paratranz.cn/api/projects/{project_id}/strings/{839578692}"

headers = {
    'Authorization': auth_token,
    'Content-Type': 'application/json',
}

payload = json.dumps({
   "stage": 0,
   "translation": "测一下API",
})

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)
pprint(response.json())
print(response.json()['original'])

200
{'context': '',
 'createdAt': '2025-12-11T07:51:47.000Z',
 'fileId': 2637743,
 'history': [],
 'id': 839578692,
 'key': 'item_Desc_oct_pants_01_Shared',
 'original': 'Carrying Capacity: 1K µSCU\\n\\nOctagon embraces a classic look '
             'and makes it boldly modern with the Altimont. These jodhpur '
             'style pants feature a slightly flared hip that becomes snug '
             'under the knees thanks to built-in garters. A uniquely shaped '
             'front seam secured by silver clips on the waist and shin provide '
             "further detailing that's both subtle and stylish. ",
 'project': 17306,
 'stage': 0,
 'translation': '测一下API',
 'uid': 66465,
 'updatedAt': '2025-12-23T04:59:02.845Z',
 'version': 6193182,
 'words': 59}
Carrying Capacity: 1K µSCU\n\nOctagon embraces a classic look and makes it boldly modern with the Altimont. These jodhpur style pants feature a slightly flared hip that becomes snug under the knees thanks to built-in garters. A uniquel

In [ ]:
class ParatranzFiller():
    
    # def __create_session(
    #     self,
    #     retries=3,
    #     backoff_factor=0.5,
    #     status_forcelist=(500, 502, 503, 504),
    #     allowed_methods=None
    # ):
    #     """
    #     创建带自动重试的 requests Session
        
    #     Args:
    #         retries: 最大重试次数
    #         backoff_factor: 重试间隔因子 (sleep = backoff_factor * (2 ^ (retry_count - 1)))
    #         status_forcelist: 需要重试的 HTTP 状态码
    #         allowed_methods: 允许重试的 HTTP 方法，None 表示所有方法
        
    #     Returns:
    #         配置好的 requests.Session 对象
    #     """
    #     session = requests.Session()
        
    #     retry = Retry(
    #         total=retries,
    #         read=retries,
    #         connect=retries,
    #         backoff_factor=backoff_factor,
    #         status_forcelist=status_forcelist,
    #         allowed_methods=allowed_methods or ['HEAD', 'GET', 'PUT', 'DELETE', 'OPTIONS', 'TRACE', 'POST']
    #     )
        
    #     adapter = HTTPAdapter(max_retries=retry)
    #     session.mount('http://', adapter)
    #     session.mount('https://', adapter)
        
    #     return session
    
    def __init__(self, proj_id: int | str, auth_token: str, page_size: int = 100, base_url: str = 'https://paratranz.cn/api') -> None:
        self.proj_id    = proj_id
        self.auth_token = auth_token
        self.page_size  = page_size
        self.base_url   = base_url
        
        # self.session = self.__create_session(5)
        self.replacer   = DescTemplateReplacer()
        self.target_ids = set()
        self.data = dict()
    
    def __proc_page(self, results):
        for r in results:
            if re.match(r'^item_desc.*', str(r['key']).lower()) is None:
                continue
            self.target_ids.add(r['id'])
            self.data[r['id']] = r
    
    def get_todo_desc(self, api_path: str = '/projects/{projectId}/strings'):
        url = self.base_url + api_path.format(projectId = self.proj_id)
        headers = {'Authorization': auth_token}
        params = {
            'stage': 0, # 指定未翻译的
            'page': 1,
            'pageSize': self.page_size,
        }
        response = requests.request("GET", url, headers=headers, params=params)
        response.raise_for_status()
        total_page = response.json()['pageCount']
        logging.info(f"共约{self.page_size * total_page}条未翻译条目，开始筛选描述条目")
        
        self.target_ids = set()
        self.data = dict()
        for page in range(total_page):
            params['page'] = page+1
            response = requests.request("GET", url, headers=headers, params=params)
            response.raise_for_status()
            self.__proc_page(response.json()['results'])
        
        logging.info(f"共{len(self.target_ids)}条未翻译的描述")
        return self.target_ids
    
    def run(self, api_path: str = '/projects/{projectId}/strings/{stringId}'):
        get_headers = {
            'Authorization': auth_token
        }
        put_headers = {
            'Content-Type': 'application/json',
            'Authorization': auth_token
        }
        
        if len(self.target_ids) == 0:
            self.get_todo_desc()
        
        count = 0
        for id in self.target_ids:
            logging.info(f"开始处理 id: {id}, key: {self.data[id]['key']}")
            url = self.base_url + api_path.format(projectId=self.proj_id, stringId=id)
            response = requests.request("GET", url, headers=get_headers)
            response.raise_for_status()
            
            if int(response.json()['stage']) != 0:
                logging.info(f"ID{id} 已经更新，跳过")
                continue
            translation = self.replacer.replace(response.json()['original'])
            if translation == '':
                logging.info(f"ID{id} 无匹配，跳过")
                continue
            
            payload = json.dumps({
                "stage": 0,
                "translation": translation
            })
            logging.debug(payload)
            response = requests.request("PUT", url, headers=put_headers, data=payload)
            response.raise_for_status()
            logging.info(f"ID{id}匹配成功，已填充：{response.json()['translation']}")
            count += 1
        
        return count

In [ ]:
import re
import json
import logging
import requests, yaml

import src.config as config
from module.replacer import DescTemplateReplacer

logging.basicConfig(level=logging.INFO, force=True)

with open('config.yaml', 'r', -1, config.ENCODE) as file:
    config_data = yaml.load(file, Loader=yaml.FullLoader)
    proj_id     = config_data['paratranz']['proj_id']
    auth_token  = config_data['paratranz']['token']

filler = ParatranzFiller(proj_id=proj_id, auth_token=auth_token)
filler.run()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): paratranz.cn:443
DEBUG:urllib3.connectionpool:https://paratranz.cn:443 "GET /api/projects/17306/strings?stage=0&page=1&pageSize=100 HTTP/1.1" 200 None
INFO:root:共约100条未翻译条目，开始筛选描述条目
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): paratranz.cn:443
DEBUG:urllib3.connectionpool:https://paratranz.cn:443 "GET /api/projects/17306/strings?stage=0&page=1&pageSize=100 HTTP/1.1" 200 None
INFO:root:共2条未翻译的描述
INFO:root:开始处理 id: 839578698, key: item_desc_ExecHangar_CompBoard_06
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): paratranz.cn:443
DEBUG:urllib3.connectionpool:https://paratranz.cn:443 "GET /api/projects/17306/strings/839578698 HTTP/1.1" 200 444
INFO:root:ID839578698 无匹配，跳过
INFO:root:开始处理 id: 839578764, key: item_desc_FLBL_ANVL_Pisces_Speed
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): paratranz.cn:443
DEBUG:urllib3.connectionpool:https://paratranz.cn:443 "GET /api/projects/173

1